In [84]:

import time
import re
import numpy as np
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By

In [85]:
country_dict = {
    "United States of America": "US",
    "Australia": "AU",
    "Canada": "CA",
    "China": "CN",
    "France": "FR",
    "Germany": "DE",
    "United Kingdom": "GB",
    "Italy": "IT",
    "Japan": "JP",
    "South Korea": "KR",
    "Russia": "RU",
    "Algeria": "DZ",
    "Angola": "AO",
    "Argentina": "AR",
    "Austria": "AT",
    "Azerbaijan": "AZ",
    "Bahrain": "BH",
    "Barbados": "BB",
    "Belarus": "BY",
    "Belgium": "BE",
    "Bermuda": "BM",
    "Brazil": "BR",
    "Bulgaria": "BG",
    "Chile": "CL",
    "Colombia": "CO",
    "Costa Rica": "CR",
    "Croatia": "HR",
    "Cyprus": "CY",
    "Czech Republic": "CZ",
    "Denmark": "DK",
    "Dominican Republic": "DO",
    "Ecuador": "EC",
    "Egypt": "EG",
    "El Salvador": "SV",
    "Finland": "FI",
    "Ghana": "GH",
    "Greece": "GR",
    "Guatemala": "GT",
    "Hong Kong": "HK",
    "Hungary": "HU",
    "India": "IN",
    "Indonesia": "ID",
    "Ireland": "IE",
    "Israel": "IL",
    "Kazakhstan": "KZ",
    "Kenya": "KE",
    "Kuwait": "KW",
    "Lebanon": "LB",
    "Lithuania": "LT",
    "Luxembourg": "LU",
    "Macau": "MO",
    "Madagascar": "MG",
    "Malaysia": "MY",
    "Malta": "MT",
    "Mexico": "MX",
    "Netherlands": "NL",
    "New Zealand": "NZ",
    "Nigeria": "NG",
    "Norway": "NO",
    "Oman": "OM",
    "Pakistan": "PK",
    "Panama": "PA",
    "Peru": "PE",
    "Philippines": "PH",
    "Poland": "PL",
    "Portugal": "PT",
    "Qatar": "QA",
    "Romania": "RO",
    "Saudi Arabia": "SA",
    "Serbia": "RS",
    "Singapore": "SG",
    "Slovakia": "SK",
    "Slovenia": "SI",
    "South Africa": "ZA",
    "Spain": "ES",
    "Sri Lanka": "LK",
    "Sweden": "SE",
    "Switzerland": "CH",
    "Taiwan": "TW",
    "Thailand": "TH",
    "Tunisia": "TN",
    "Turkey": "TR",
    "Ukraine": "UA",
    "United Arab Emirates": "AE",
    "Uruguay": "UY",
    "Uzbekistan": "UZ",
    "Venezuela": "VE",
    "Vietnam": "VN",
    "Bolivia": "BO",
    "Cambodia": "KH",
    "Estonia": "EE",
    "Latvia": "LV",
    "Nicaragua": "NI",
    "Paraguay": "PY",
    "Afghanistan": "AF",
    "Georgia": "GE",
    "Iraq": "IQ",
    "Libya": "LY",
    "Morocco": "MA",
    "Mozambique": "MZ",
    "Myanmar": "MM",
    "Yemen": "YE"
}


dates = ['2024-12-30', '2024-12-31']

categories = {
    'Entertainment': '6016',
    'Education': '6017'
}

base_url = 'https://app.sensortower.com/top-charts?os=ios&device=iphone&'


In [87]:
url = base_url + f'country={country_dict['Yemen']}&category={categories['Education']}&date={dates[0]}'
driver = webdriver.Chrome()
driver.maximize_window()
driver.get(url)
time.sleep(1)
rows = driver.find_element(By.TAG_NAME, 'table').find_element(By.TAG_NAME, 'tbody').find_elements(By.TAG_NAME, 'tr')

for idx, row in enumerate(rows):
    data = row.find_elements(By.TAG_NAME, 'td')
    paid_app_data = data[2].find_element(By.TAG_NAME, 'span').find_element(By.TAG_NAME, 'div').find_element(By.TAG_NAME, 'div')
    data = paid_app_data.text.split('\n')
    rating_data = paid_app_data.find_elements(By.TAG_NAME, 'div')[1].find_element(By.XPATH, '//span[starts-with(@aria-label, "Average Rating")]')
    aria_label_value = rating_data.get_attribute('aria-label')
    match = re.search(r"(\d+\.\d+)", aria_label_value)
    print(f'Rank: {idx + 1}')
    print('name: ' + data[0])
    print('affiliation: ' + data[1])
    if match:
        rating = float(match.group(1))
        print('Rating: ' + f'{rating}')
    print('number of rating: ' + f'{int(data[2].strip("()"))}')
    print('-----------------------------------------------------------------------------------------------------')

driver.quit()

Rank: 1
name: DarkGPT ask-AI
affiliation: Mohamed amine El Bouazzi
Rating: 4.61
number of rating: 0
-----------------------------------------------------------------------------------------------------
Rank: 2
name: تعلم اللغة الانجليزية
affiliation: mawuood alghzali
Rating: 4.61
number of rating: 1
-----------------------------------------------------------------------------------------------------
Rank: 3
name: حفظ القرآن - إصدارة كاملة
affiliation: BIGITEC GmbH
Rating: 4.61
number of rating: 1
-----------------------------------------------------------------------------------------------------
Rank: 4
name: AnkiMobile Flashcards
affiliation: Ankitects Pty Ltd
Rating: 4.61
number of rating: 1
-----------------------------------------------------------------------------------------------------
Rank: 5
name: تعلم اللغة التركية
affiliation: mawuood alghzali
Rating: 4.61
number of rating: 0
--------------------------------------------------------------------------------------------------